# Yahoo Finance News Web Scraper
Notes: 
* Continued issues accessing articles behind a paywall or external source (e.g., https://finance.yahoo.com/news/top-midday-stories-pepsico-buy-160405890.html)

In [187]:
import re
import csv
from time import sleep
from bs4 import BeautifulSoup
import requests
import yfinance as yf

In [188]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import math
import re
# import openpyxl

# import requests
# TODO: Compare using requests vs using drivers

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

from seleniumbase import Driver

In [189]:
# Set up web driver
options = webdriver.ChromeOptions()
driver = Driver(uc=True, incognito=True)

In [190]:
# Get website html data 
def get_basesoup(driver, url, wait=False, until_class='ClassOfMyElement'):
    driver.get(url)

    # Wait for page and element to completely load
    if wait:
        delay = 3 # seconds
        try:
            WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.CLASS_NAME, until_class)))
        except TimeoutException:
            print("Loading took too much time!")
    
    basesoup=BeautifulSoup(driver.page_source,"html.parser")
    return basesoup

In [191]:
def get_news_info(url):
    '''
    Return a list of strings for a given url, 
    where each string is a sentence in the linked article.
    '''
    soup = get_basesoup(driver, url)

    try:
        article = soup.find("div", class_="article-wrap no-bb")
        
        if not article:
            print(f"No articles found on page {url}")
            return []

        # cover_wrap = article.find("div", class_="cover-wrap yf-1p8y0lh")
        # title = cover_wrap.find("h1", class_="cover-title yf-1p8y0lh")
             
        body_wrap = article.find("div", class_="body-wrap yf-i23rhs")
        body = body_wrap.find("div", class_="body yf-5ef8bf")
        text = body.find_all("p", class_="yf-1pe5jgt")
        # for paragraph in text:
        #     print(paragraph.text.strip())
        
        return [paragraph.text.strip() for paragraph in text]
            
        
    except:
        print(f"Error accessing articles on page {url}")
        return []
    
    

In [192]:
# Input company
ticker = 'TSLA' # Currently hard-coded. Replace with user input.

# Access company news
news = yf.Ticker(ticker).news

# # Article is a dictionary, info is a key value pairing
# for article in news:
#     [print(info) for info in article.items()]

# Construct list of urls
urls = {dictionary['link'] for dictionary in news}   
urls

{'https://finance.yahoo.com/m/29c92e28-b5ab-392e-8f22-ae525864f9e8/tesla-earnings-due-after.html',
 'https://finance.yahoo.com/m/4d8cdde7-a669-3553-b6b6-a30499bdc041/bad-evs%2C-a-giant-meat-recall%2C.html',
 'https://finance.yahoo.com/m/632b6af3-08a3-389b-b46a-4e29e4c056cb/3-reasons-rivian-is-the.html',
 'https://finance.yahoo.com/m/dd849627-6fd9-3fb5-8c4d-0b7520e45713/tesla-executive-attacks.html',
 'https://finance.yahoo.com/m/eeafd800-e38b-392e-ace5-c38f67faa5e8/tesla-stock-falls-after.html',
 'https://finance.yahoo.com/m/fe9aaa97-1fee-3520-9247-c47d4a5bf9a2/elon-musk%27s-tesla-robot.html',
 'https://finance.yahoo.com/news/elon-musk-spent-teen-years-143036203.html',
 'https://finance.yahoo.com/news/nhtsa-opens-defect-investigation-tesla-122206336.html'}

In [193]:
# Scrape one article
get_news_info("https://finance.yahoo.com/news/top-stock-movers-now-apa-172011005.html")

['',
 'Major U.S. indexes lost ground at midday Tuesday as worries about rising tensions in the Middle East weighed on markets.',
 'Oil prices rose, lifting shares of APA and other companies in the oil industry.',
 'Apple shares fell as Barclays warned of weak demand for the iPhone 16.',
 '',
 'Major U.S. indexes lost ground to begin the new month and quarter at midday Tuesday as concerns about Middle East tensions rattled the market. The Nasdaq fell more than 2%, the S&P 500 lost 1%, and the Dow was lower as well.',
 'Shares of APA Corporation (APA) and other oil companies advanced along with the price of crude on concerns about tensions in the Middle East.',
 'The prospect of escalating conflict in the region also lifted shares of military contractors Northrop Grumman (NOC) and Lockheed Martin (LMT).',
 'Tesla (TSLA) shares declined as Chinese electric vehicle (EV) rivals posted record September deliveries and Apple (AAPL) shares fell as Barclays warned of weak demand for the iPhone 

In [194]:
# Scrape all related articles
for url in urls:
    article_text = get_news_info(url)
    print(article_text)

['About 10% of electric vehicle owners regret their purchases, according to a new survey, though the figure varies significantly depending on the automaker.', 'Read More', 'Major retailers, including Amazon (AMZN), Costco (COST), Walmart (WMT), and Trader Joe’s, have issued a nationwide recall for over 300 ready-to-eat meals due to a potential outbreak of Listeria monocytogenes.', 'Read More', 'Amazon (AMZN) is joining its artificial intelligence competitors in the race for nuclear energy.', 'Read More', 'When deciding where to live in the U.S., many people take safety into consideration – not only for their families but for themselves.', 'Read More', 'Since releasing its Mate 60 Pro series last August, Chinese tech giant Huawei has made a comeback in China’s smartphone market, knocking Apple (AAPL) from its pedestal.', 'Read More', 'The U.S. economy remains a heated topic in presidential debates and a top concern for American voters. Eight in ten registered voters say it will be cruci

In [195]:
driver.quit()